# Demo: Knowledge Graph Completion

In this tutorial demo, we will use the Graph4NLP library to build a GNN-based knowledge graph completion model. The model consists of

+ graph embedding module (e.g., GGNN)
+ predictoin module (e.g., DistMult decoder)

We will use the built-in module APIs to build the model, and evaluate it on the Kinship dataset.

## Environment setup

Please follow the instructions [here](https://github.com/graph4ai/graph4nlp_demo#environment-setup) to set up the environment. Please also run the following commands to install extra packages used in this demo.
```
pip install spacy
python -m spacy download en_core_web_sm
pip install h5py
pip install future
```

# Data preprocessing for KGC
+ Run the preprocessing script for WN18RR and Kinship: ```sh kg_completion/preprocess.sh```
+ You can now run the model

+ This notebook is tested on :

```
torch == 1.8.0
torchtext == 0.9.0
spacy == 3.0.7
graph4nlp == 0.4.1 # macos, cpu
```

In [2]:
import torch
import numpy as np
import torch.backends.cudnn as cudnn

from evaluation import ranking_and_hits
from model import ConvE, Distmult, Complex, GGNNDistMult, GCNDistMult, GCNComplex

from spodernet.preprocessing.pipeline import DatasetStreamer
from spodernet.preprocessing.processors import JsonLoaderProcessors, Tokenizer, AddToVocab, SaveLengthsToState, StreamToHDF5, SaveMaxLengthsToState, CustomTokenizer
from spodernet.preprocessing.processors import ConvertTokenToIdx, ApplyFunction, ToLower, DictKey2ListMapper, ApplyFunction, StreamToBatch
from spodernet.utils.global_config import Config, Backends
from spodernet.utils.logger import Logger, LogLevel
from spodernet.preprocessing.batching import StreamBatcher
from spodernet.preprocessing.pipeline import Pipeline
from spodernet.preprocessing.processors import TargetIdx2MultiTarget
from spodernet.hooks import LossHook, ETAHook
from spodernet.utils.util import Timer
from spodernet.preprocessing.processors import TargetIdx2MultiTarget
import argparse

np.set_printoptions(precision=3)
cudnn.benchmark = True

In [3]:
''' Preprocess knowledge graph using spodernet. '''
def preprocess(dataset_name, delete_data=False):
    full_path = 'data/{0}/e1rel_to_e2_full.json'.format(dataset_name)
    train_path = 'data/{0}/e1rel_to_e2_train.json'.format(dataset_name)
    dev_ranking_path = 'data/{0}/e1rel_to_e2_ranking_dev.json'.format(dataset_name)
    test_ranking_path = 'data/{0}/e1rel_to_e2_ranking_test.json'.format(dataset_name)

    keys2keys = {}
    keys2keys['e1'] = 'e1' # entities
    keys2keys['rel'] = 'rel' # relations
    keys2keys['rel_eval'] = 'rel' # relations
    keys2keys['e2'] = 'e1' # entities
    keys2keys['e2_multi1'] = 'e1' # entity
    keys2keys['e2_multi2'] = 'e1' # entity
    input_keys = ['e1', 'rel', 'rel_eval', 'e2', 'e2_multi1', 'e2_multi2']
    d = DatasetStreamer(input_keys)
    d.add_stream_processor(JsonLoaderProcessors())
    d.add_stream_processor(DictKey2ListMapper(input_keys))

    # process full vocabulary and save it to disk
    d.set_path(full_path)
    p = Pipeline(args.data, delete_data, keys=input_keys, skip_transformation=True)
    p.add_sent_processor(ToLower())
    p.add_sent_processor(CustomTokenizer(lambda x: x.split(' ')),keys=['e2_multi1', 'e2_multi2'])
    p.add_token_processor(AddToVocab())
    p.execute(d)
    p.save_vocabs()


    # process train, dev and test sets and save them to hdf5
    p.skip_transformation = False
    for path, name in zip([train_path, dev_ranking_path, test_ranking_path], ['train', 'dev_ranking', 'test_ranking']):
        d.set_path(path)
        p.clear_processors()
        p.add_sent_processor(ToLower())
        p.add_sent_processor(CustomTokenizer(lambda x: x.split(' ')),keys=['e2_multi1', 'e2_multi2'])
        p.add_post_processor(ConvertTokenToIdx(keys2keys=keys2keys), keys=['e1', 'rel', 'rel_eval', 'e2', 'e2_multi1', 'e2_multi2'])
        p.add_post_processor(StreamToHDF5(name, samples_per_file=1000, keys=input_keys))
        p.execute(d)

In [4]:
def main(args, model_path):
    if args.preprocess:
        preprocess(args.data, delete_data=True)
    input_keys = ['e1', 'rel', 'rel_eval', 'e2', 'e2_multi1', 'e2_multi2']
    p = Pipeline(args.data, keys=input_keys)
    p.load_vocabs()
    vocab = p.state['vocab']

    train_batcher = StreamBatcher(args.data, 'train', args.batch_size, randomize=True, keys=input_keys, loader_threads=args.loader_threads)
    dev_rank_batcher = StreamBatcher(args.data, 'dev_ranking', args.test_batch_size, randomize=False, loader_threads=args.loader_threads, keys=input_keys)
    test_rank_batcher = StreamBatcher(args.data, 'test_ranking', args.test_batch_size, randomize=False, loader_threads=args.loader_threads, keys=input_keys)


    data = []
    rows = []
    columns = []
    num_entities = vocab['e1'].num_token
    num_relations = vocab['rel'].num_token

    if args.preprocess:
        for i, str2var in enumerate(train_batcher):
            print("batch number:", i)
            for j in range(str2var['e1'].shape[0]):
                for k in range(str2var['e2_multi1'][j].shape[0]):
                    if str2var['e2_multi1'][j][k] != 0:
                        data.append(str2var['rel'][j].cpu().tolist()[0])
                        rows.append(str2var['e1'][j].cpu().tolist()[0])
                        columns.append(str2var['e2_multi1'][j][k].cpu().tolist())
                    else:
                        break

        from graph4nlp.pytorch.data.data import GraphData, to_batch
        KG_graph = GraphData()
        KG_graph.add_nodes(num_entities)
        for e1, rel, e2 in zip(rows, data, columns):
            KG_graph.add_edge(e1, e2)
            eid = KG_graph.edge_ids(e1, e2)[0]
            KG_graph.edge_attributes[eid]['token'] = rel

        torch.save(KG_graph, '{}/processed/KG_graph.pt'.format(args.data))
        return


    if args.model is None:
        model = ConvE(args, vocab['e1'].num_token, vocab['rel'].num_token)
    elif args.model == 'conve':
        model = ConvE(args, vocab['e1'].num_token, vocab['rel'].num_token)
    elif args.model == 'distmult':
        model = Distmult(args, vocab['e1'].num_token, vocab['rel'].num_token)
    elif args.model == 'complex':
        model = Complex(args, vocab['e1'].num_token, vocab['rel'].num_token)
    elif args.model == 'ggnn_distmult':
        model = GGNNDistMult(args, vocab['e1'].num_token, vocab['rel'].num_token)
    elif args.model == 'gcn_distmult':
        model = GCNDistMult(args, vocab['e1'].num_token, vocab['rel'].num_token)
    elif args.model == 'gcn_complex':
        model = GCNComplex(args, vocab['e1'].num_token, vocab['rel'].num_token)
    else:
        raise Exception("Unknown model!")

    if args.model in ['ggnn_distmult', 'gcn_distmult', 'gcn_complex']:
        graph_path = '{}/processed/KG_graph.pt'.format(args.data)
        KG_graph = torch.load(graph_path)
        if Config.cuda:
            KG_graph = KG_graph.to('cuda')
    else:
        KG_graph = None

    train_batcher.at_batch_prepared_observers.insert(1,TargetIdx2MultiTarget(num_entities, 'e2_multi1', 'e2_multi1_binary'))

    eta = ETAHook('train', print_every_x_batches=args.log_interval)
    train_batcher.subscribe_to_events(eta)
    train_batcher.subscribe_to_start_of_epoch_event(eta)
    train_batcher.subscribe_to_events(LossHook('train', print_every_x_batches=args.log_interval))
    if Config.cuda:
        model.cuda()
    if args.resume:
        model_params = torch.load(model_path)
        print(model)
        total_param_size = []
        params = [(key, value.size(), value.numel()) for key, value in model_params.items()]
        for key, size, count in params:
            total_param_size.append(count)
            print(key, size, count)
        print(np.array(total_param_size).sum())
        model.load_state_dict(model_params)
        model.eval()
        ranking_and_hits(model, test_rank_batcher, vocab, 'test_evaluation', kg_graph=KG_graph)
        ranking_and_hits(model, dev_rank_batcher, vocab, 'dev_evaluation', kg_graph=KG_graph)
    else:
        model.init()

    total_param_size = []
    params = [value.numel() for value in model.parameters()]
    print(params)
    print(np.sum(params))

    best_mrr = 0

    opt = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    for epoch in range(args.epochs):
        model.train()
        for i, str2var in enumerate(train_batcher):
            opt.zero_grad()
            e1 = str2var['e1']
            rel = str2var['rel']
            e2_multi = str2var['e2_multi1_binary'].float()
            # label smoothing
            e2_multi = ((1.0-args.label_smoothing)*e2_multi) + (1.0/e2_multi.size(1))

            pred = model.forward(e1, rel, KG_graph)
            loss = model.loss(pred, e2_multi)
            loss.backward()
            opt.step()

            train_batcher.state.loss = loss.cpu()

        model.eval()
        with torch.no_grad():
            if epoch % 2 == 0 and epoch > 0:
                dev_mrr = ranking_and_hits(model, dev_rank_batcher, vocab, 'dev_evaluation', kg_graph=KG_graph)
                if dev_mrr > best_mrr:
                    best_mrr = dev_mrr
                    print('saving best model to {0}'.format(model_path))
                    torch.save(model.state_dict(), model_path)
            if epoch % 2 == 0:
                if epoch > 0:
                    ranking_and_hits(model, test_rank_batcher, vocab, 'test_evaluation', kg_graph=KG_graph)

### Config Setup

In [5]:
parser = argparse.ArgumentParser(description='Link prediction for knowledge graphs')
parser.add_argument('--batch-size', type=int, default=128, help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=128, help='input batch size for testing/validation (default: 128)')
parser.add_argument('--epochs', type=int, default=1000, help='number of epochs to train (default: 1000)')
parser.add_argument('--lr', type=float, default=0.003, help='learning rate (default: 0.003)')
parser.add_argument('--seed', type=int, default=1234, metavar='S', help='random seed (default: 17)')
parser.add_argument('--log-interval', type=int, default=100, help='how many batches to wait before logging training status')
parser.add_argument('--data', type=str, default='kinship', help='Dataset to use: {FB15k-237, YAGO3-10, WN18RR, umls, nations, kinship}, default: FB15k-237')
parser.add_argument('--l2', type=float, default=0.0, help='Weight decay value to use in the optimizer. Default: 0.0')
parser.add_argument('--model', type=str, default='ggnn_distmult', help='Choose from: {conve, distmult, complex}')
parser.add_argument('--direction_option', type=str, default='undirected', help='Choose from: {undirected, bi_sep, bi_fuse}')
parser.add_argument('--embedding-dim', type=int, default=200, help='The embedding dimension (1D). Default: 200')
parser.add_argument('--embedding-shape1', type=int, default=20, help='The first dimension of the reshaped 2D embedding. The second dimension is infered. Default: 20')
parser.add_argument('--hidden-drop', type=float, default=0.25, help='Dropout for the hidden layer. Default: 0.3.')
parser.add_argument('--input-drop', type=float, default=0.2, help='Dropout for the input embeddings. Default: 0.2.')
parser.add_argument('--feat-drop', type=float, default=0.2, help='Dropout for the convolutional features. Default: 0.2.')
parser.add_argument('--lr-decay', type=float, default=0.995, help='Decay the learning rate by this factor every epoch. Default: 0.995')
parser.add_argument('--loader-threads', type=int, default=4, help='How many loader threads to use for the batch loaders. Default: 4')
parser.add_argument('--preprocess', action='store_true', help='Preprocess the dataset. Needs to be executed only once. Default: 4')
parser.add_argument('--resume', action='store_true', help='Resume a model.')
parser.add_argument('--use-bias', action='store_true', help='Use a bias in the convolutional layer. Default: True')
parser.add_argument('--label-smoothing', type=float, default=0.1, help='Label smoothing value to use. Default: 0.1')
parser.add_argument('--hidden-size', type=int, default=9728, help='The side of the hidden layer. The required size changes with the size of the embeddings. Default: 9728 (embedding size 200).')

parser.add_argument('--channels', type=int, default=200, help='The side of the hidden layer. The required size changes with the size of the embeddings. Default: 9728 (embedding size 200).')
parser.add_argument('--kernel_size', type=int, default=5, help='The side of the hidden layer. The required size changes with the size of the embeddings. Default: 9728 (embedding size 200).')

_StoreAction(option_strings=['--kernel_size'], dest='kernel_size', nargs=None, const=None, default=5, type=<class 'int'>, choices=None, help='The side of the hidden layer. The required size changes with the size of the embeddings. Default: 9728 (embedding size 200).', metavar=None)

### If you run the task for the first time, run with:

In [6]:
args = parser.parse_args(args=['--data', 'kinship', '--model', 'ggnn_distmult', '--preprocess'])

In [7]:
# parse console parameters and set global variables
Config.backend = 'pytorch'
Config.cuda = False
Config.embedding_dim = args.embedding_dim

model_name = '{2}_{0}_{1}'.format(args.input_drop, args.hidden_drop, args.model)
model_path = 'saved_models/{0}_{1}.model'.format(args.data, model_name)

torch.manual_seed(args.seed)

In [8]:
args.preprocess

True

In [9]:
import warnings
warnings.filterwarnings('ignore')

### After preprocess the kinship data, then run:

In [10]:
main(args, model_path)

2021-08-11 01:25:45.025464 (WARNING): delete_all_previous_data=True! Deleting all folder contents of folder /Users/hugo/.data/kinship!
2021-08-11 01:25:45.070188 (INFO): Recreating path: /Users/hugo/.data/kinship
2021-08-11 01:25:45.193284 (INFO): Time taken for 10000 samples for input type rel for processor AddToVocab: 0.02 seconds
2021-08-11 01:25:45.372957 (INFO): Time taken for 10000 samples for input type e2 for processor ToLower: 0.01 seconds
2021-08-11 01:25:45.373351 (INFO): Time taken for 10000 samples for input type e2 for processor SaveLengthsToState: 0.04 seconds
2021-08-11 01:25:45.729156 (INFO): Time taken for 10000 samples for input type e2_multi2 for processor CustomTokenizer: 0.02 seconds
2021-08-11 01:25:45.817601 (INFO): Saving vocab to: /Users/hugo/.data/kinship/vocab
2021-08-11 01:25:45.818879 (INFO): Saving vocab to: /Users/hugo/.data/kinship/vocab_e1
2021-08-11 01:25:45.819535 (INFO): Saving vocab to: /Users/hugo/.data/kinship/vocab_rel
2021-08-11 01:25:45.820164

In [11]:
args = parser.parse_args(args=['--data', 'kinship', '--model', 'ggnn_distmult'])

In [ ]:
main(args, model_path)

2021-08-11 01:26:36.991522 (WARNING): Pipeline path /Users/hugo/.data/kinship already exist. This pipeline may overwrite data in this path!
2021-08-11 01:26:36.992632 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab
2021-08-11 01:26:36.993748 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab_e1
2021-08-11 01:26:37.002026 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab_rel
2021-08-11 01:26:37.003720 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab_rel_eval
2021-08-11 01:26:37.005099 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab_e2
2021-08-11 01:26:37.006257 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab_e2_multi1
2021-08-11 01:26:37.012940 (INFO): Loading vocab from: /Users/hugo/.data/kinship/vocab_e2_multi2
[21200, 10400, 40000, 200, 120000, 120000, 600, 600]
313000
2021-08-11 01:26:58.652849 (INFO): Total epoch time: 0:00:16
2021-08-11 01:26:58.695876 (INFO): 

2021-08-11 01:26:58.697975 (INFO): ###################

2021-08-11 01:29:27.287915 (INFO): Hits left @1: 0.0322265625
2021-08-11 01:29:27.288484 (INFO): Hits right @1: 0.029296875
2021-08-11 01:29:27.288765 (INFO): Hits @1: 0.03076171875
2021-08-11 01:29:27.288944 (INFO): Hits left @2: 0.0625
2021-08-11 01:29:27.289116 (INFO): Hits right @2: 0.0576171875
2021-08-11 01:29:27.289372 (INFO): Hits @2: 0.06005859375
2021-08-11 01:29:27.290036 (INFO): Hits left @3: 0.0947265625
2021-08-11 01:29:27.290208 (INFO): Hits right @3: 0.080078125
2021-08-11 01:29:27.290460 (INFO): Hits @3: 0.08740234375
2021-08-11 01:29:27.290628 (INFO): Hits left @4: 0.126953125
2021-08-11 01:29:27.290795 (INFO): Hits right @4: 0.1025390625
2021-08-11 01:29:27.291381 (INFO): Hits @4: 0.11474609375
2021-08-11 01:29:27.291551 (INFO): Hits left @5: 0.154296875
2021-08-11 01:29:27.291719 (INFO): Hits right @5: 0.1328125
2021-08-11 01:29:27.291970 (INFO): Hits @5: 0.1435546875
2021-08-11 01:29:27.292137 (INFO): Hits left @6: 0.1806640625
2021-08-11 01:29:27.292691 (INFO): Hi

2021-08-11 01:31:19.573263 (INFO): Hits left @1: 0.0810546875
2021-08-11 01:31:19.574288 (INFO): Hits right @1: 0.078125
2021-08-11 01:31:19.574632 (INFO): Hits @1: 0.07958984375
2021-08-11 01:31:19.575246 (INFO): Hits left @2: 0.1533203125
2021-08-11 01:31:19.575433 (INFO): Hits right @2: 0.162109375
2021-08-11 01:31:19.575697 (INFO): Hits @2: 0.15771484375
2021-08-11 01:31:19.575870 (INFO): Hits left @3: 0.23828125
2021-08-11 01:31:19.576041 (INFO): Hits right @3: 0.2373046875
2021-08-11 01:31:19.576340 (INFO): Hits @3: 0.23779296875
2021-08-11 01:31:19.576512 (INFO): Hits left @4: 0.3193359375
2021-08-11 01:31:19.576682 (INFO): Hits right @4: 0.3125
2021-08-11 01:31:19.576935 (INFO): Hits @4: 0.31591796875
2021-08-11 01:31:19.577107 (INFO): Hits left @5: 0.3955078125
2021-08-11 01:31:19.577277 (INFO): Hits right @5: 0.3779296875
2021-08-11 01:31:19.577531 (INFO): Hits @5: 0.38671875
2021-08-11 01:31:19.578088 (INFO): Hits left @6: 0.4541015625
2021-08-11 01:31:19.578281 (INFO): Hits

2021-08-11 01:32:53.864102 (INFO): Total epoch time: 0:00:10
2021-08-11 01:32:53.865255 (INFO): 

2021-08-11 01:32:53.899978 (INFO): ########################################
2021-08-11 01:32:53.921692 (INFO):           COMPLETED EPOCH: 10                              
2021-08-11 01:32:53.944452 (INFO): train Loss: 0.16465	99% CI: (0.1496, 0.17971), n=23
2021-08-11 01:32:53.944538 (INFO): ########################################
2021-08-11 01:32:53.944586 (INFO): 

2021-08-11 01:33:06.240679 (INFO): Total epoch time: 0:00:12
2021-08-11 01:33:06.241700 (INFO): 

2021-08-11 01:33:06.241780 (INFO): ########################################
2021-08-11 01:33:06.241828 (INFO):           COMPLETED EPOCH: 11                              
2021-08-11 01:33:06.241868 (INFO): train Loss: 0.1608	99% CI: (0.14711, 0.17449), n=23
2021-08-11 01:33:06.241904 (INFO): ########################################
2021-08-11 01:33:06.241940 (INFO): 

2021-08-11 01:33:06.242115 (INFO): 
2021-08-11 01:33:06.266511